# Perplexity Pytorch Implemention

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
torch.manual_seed(0)

In [22]:
# config
dim = 16
vocab_size = 32
seq_len = 5
bs = 2 # batch size

In [23]:
class ToyModel(nn.Module):
    def __init__(self, dim, vocab_size):
        super(ToyModel, self).__init__()
        self.embd = nn.Embedding(vocab_size, dim)
        self.llama = nn.Linear(dim, dim)
        self.lm_head = nn.Linear(dim, vocab_size)
    def forward(self, x):
        embd = self.embd(x)
        hidden = self.llama(embd)
        logits = self.lm_head(hidden)
        return logits

model = ToyModel(dim, vocab_size)
print(model)

ToyModel(
  (embd): Embedding(32, 16)
  (llama): Linear(in_features=16, out_features=16, bias=True)
  (lm_head): Linear(in_features=16, out_features=32, bias=True)
)

In [24]:
# 创建数据
x = torch.randint(high=vocab_size, size=(bs, seq_len))
print('input id:\n', x)
y = x[:,1:]  
x = x[:,:-1]
print('batch input:\n', x)
print('batch label:\n', y)

input id:
 tensor([[21, 20, 10, 26, 25],
        [17,  5, 25, 20, 21]])

batch input:
 tensor([[21, 20, 10, 26],
        [17,  5, 25, 20]])

batch label:
 tensor([[20, 10, 26, 25],
        [ 5, 25, 20, 21]])

In [141]:
# 计算CE loss
def CE_loss(label, logits):
    bs, seq, _ = logits.shape
    neg_logprob = -F.log_softmax(logits, dim = -1) # bs, seq, dim
    
    idx = torch.arange(seq)
    ce_loss_token = torch.zeros(bs, seq)
    
    for i in range(bs):
        ce_loss_token[i,:] = neg_logprob[i, idx, label[i,:]]
    
    ce_loss_seq = ce_loss_token.mean(dim = -1)
    
    return ce_loss_token, ce_loss_seq

def ppl(loss):
    ppl = loss.exp()
    ppl_mean = ppl.mean()
    return ppl_mean

In [142]:
logits = model(x)
print(logits.shape)

torch.Size([2, 4, 32])

In [143]:
ce_loss_token, ce_loss_seq = CE_loss(y, logits)
print(ce_loss_token)
print(ce_loss_seq)

tensor([[3.3783, 3.7758, 3.6459, 3.4241],
        [3.7209, 3.6905, 4.7364, 3.6323]], grad_fn=<CopySlices>)

tensor([3.5560, 3.9450], grad_fn=<MeanBackward1>)

In [144]:
ppl_batch = ppl(ce_loss_seq)
print(ppl_batch)

tensor(43.3506, grad_fn=<MeanBackward0>)